In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from parquet_data_reader import ParquetDataReader
from utils.process_data import  user_item_binary_interaction
parquet_reader = ParquetDataReader()

In [3]:
train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
user_df = parquet_reader.read_data("../../data/train/history.parquet")
item_df = parquet_reader.read_data("../../data/articles.parquet")

In [4]:
binary_interaction = user_item_binary_interaction(train_behavior_df, user_df, item_df)
binary_interaction

user_id,article_id,clicked
u32,i32,i32
13538,3001353,0
13538,3003065,0
13538,3012771,0
13538,3023463,0
13538,3032577,0
…,…,…
1710834,9803492,0
1710834,9803505,0
1710834,9803525,0


In [ ]:
# Count all the rows in the binary_interaction where click is 1
clicks = binary_interaction.filter(binary_interaction.click == 1).count()

ValueError: expected 3 values when selecting columns by boolean mask, got 314052606